In [22]:
import requests # type: ignore
import numpy as np # type: ignore
import re
import json
import pandas as pd
import os

In [23]:
NUM_COHORTS = 24
COHORT_SIZE = 8
NUM_ROUNDS = 75

MEMORY_DIR = "memory"
MEMORY_SIZE = 10
MODEL_NAME= "mistral"

In [24]:
NUM_COHORTS = 6
COHORT_SIZE = 4
NUM_ROUNDS = 3

In [25]:
TREATMENTS = {
	0.6: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 42),
		('B', 'R'): (42, 0),
		('B', 'B'): (12, 12)
	},
	1: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 40),
		('B', 'R'): (40, 0),
		('B', 'B'): (20, 20)
	},
	2: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 35),
		('B', 'R'): (35, 0),
		('B', 'B'): (40, 40)
	}
}

In [26]:
API_URL = "http://127.0.0.1:11434"
GENERATE_ENDPOINT = "/api/generate"
CHAT_ENDPOINT = "/api/chat"

In [27]:
rand_gen = np.random.default_rng(seed=23)

In [28]:
class Participant:
	def __init__(self, id):
		self.id = id
		self.memory = []
		self.payout = 0
		self.rounds = 0
		self.prompt = ""

	def __eq__(self, other):
		return self.id == other.id
	
	def __lt__(self, other):
		return self.id < other.id
	
	def __gt__(self, other):
		return self.id > other.id
	
	def __hash__(self):
		return int(self.id)
	
	def __repr__(self):
		return f"<Participant {{id: {self.id}}}>"
	
	def set_prompt(self, prompt):
		self.prompt = prompt
	
	def update_memory(self, ctx):
		self.memory = [*ctx]
		if len(self.memory) >= MEMORY_SIZE:
			self.memory.pop(1)
		self.save_memory()
		
	def save_memory(self):
		if not os.path.exists(MEMORY_DIR):
			os.makedirs(MEMORY_DIR)
		with open(f"{MEMORY_DIR}/{self.id}.json", "wt") as file:
			file.write(json.dumps(self.memory, indent=4))

In [29]:
class Cohort:
	def __init__(self, id, participants):
		self.id = id
		self.participants = participants
		self.treatment = None

In [30]:
participant_ids = np.arange(101, NUM_COHORTS * COHORT_SIZE + 101, dtype=int)
rand_gen.shuffle(participant_ids)
participants = [Participant(id) for id in participant_ids]

In [31]:
cohort_ids = np.arange(1, NUM_COHORTS + 1, dtype=int)
cohort_splits = np.array_split(participants, NUM_COHORTS)
cohorts = [Cohort(id, participant) for id, participant in zip(cohort_ids, cohort_splits)]

In [32]:
treatment_splits = np.array_split(cohorts, len(TREATMENTS))
for cohorts_for_treatment, treatment in zip(treatment_splits, TREATMENTS.keys()):
	for cohort in cohorts_for_treatment:
		cohort.treatment = treatment

In [33]:
def prompt_model(prompt, context):
	messages = [
		*context,
		{
			"role": "user",
			"content": prompt
		}
	]
	data = {
    "model": MODEL_NAME,
		"messages": messages,
    "stream": False
	}
	res = requests.post(API_URL + CHAT_ENDPOINT, json=data).json()
	return [*messages, res['message']]

In [34]:
def get_choice(response):
	matches = re.findall("{[RB]}", response)
	if len(matches) > 0:
		return matches[-1][1]
	else:
		matches = re.findall("[RB]", response)
		if len(matches) > 0:
			return matches[-1]
		else:
			return ""

In [35]:
def generate_start_prompt(treatment):
	return f"You are an undergraduate student participating in a lab experiment. You play a game with an anonymous player in which you simultaneously make a choice. Your payoff depends on both choices. If you both pick R, you each get {TREATMENTS[treatment][('R','R')][0]}$. If you choose R while they choose B, you get {TREATMENTS[treatment][('R','B')][0]}$, and they get {TREATMENTS[treatment][('R','B')][1]}$. Similarly, if you pick B while they pick R, you get {TREATMENTS[treatment][('B','R')][0]}$, and they get {TREATMENTS[treatment][('B','R')][1]}$. If you both pick B, you each earn {TREATMENTS[treatment][('B','B')][0]}$. The game has {NUM_ROUNDS} rounds. What's your choice? Perform reasoning as a human player. Append your choice letter in curly brackets as a last character."

In [36]:
def generate_cont_prompt(choice1, choice2, treatment):
	return f"You chose {choice1}. Your opponent chose {choice2}. Your payoff is {TREATMENTS[treatment][(choice1, choice2)][0]}$. Your opponent's payoff is {TREATMENTS[treatment][(choice1, choice2)][1]}$. Please play the next round."

In [37]:
def generate_end_prompt(choice1, choice2, treatment, payout):
	return f"You chose {choice1}. Your opponent chose {choice2}. Your payoff is {TREATMENTS[treatment][(choice1, choice2)][0]}$. Your opponent's payoff is {TREATMENTS[treatment][(choice1, choice2)][1]}$. The game is now over. Your total payoff was {payout + TREATMENTS[treatment][(choice1, choice2)][0]}$."

In [38]:
def generate_game_order(cohort):
	result = []
	participants = cohort.participants

	for _ in range(NUM_ROUNDS):
		pairs = []
		remaining_participants = participants.copy()

		while len(remaining_participants) > 1:
			i, j = rand_gen.choice(len(remaining_participants), size=2, replace=False)
			pair = (remaining_participants[i], remaining_participants[j])
			pairs.append(pair)

			remaining_participants = list(remaining_participants)
			remaining_participants.remove(pair[0])
			remaining_participants.remove(pair[1])

		if len(remaining_participants) == 1:
			pairs.append((remaining_participants[0], None))

		result.extend(pairs)

	return result

In [39]:
def save_results(filename, row):
	new_row = pd.DataFrame(row)
	if os.path.isfile(filename):
		data = pd.read_csv(filename)
		data = pd.concat([data, new_row])
		data.to_csv(filename, index=False)
	else:
		data = new_row
		data.to_csv(filename, index=False)

In [40]:
game_id = 0
total_rounds = NUM_COHORTS * COHORT_SIZE * NUM_ROUNDS // 2

In [41]:
for cohort in cohorts:
	print("----*----"*5)
	print(f"Cohort ID: {cohort.id}", end="\t")
	print(f"Cohort Treatment: {cohort.treatment}")
	print("Cohort Participants:")
	for participant in cohort.participants:
		print(participant.id, end="\t")
	print("")
	print("----*----"*5)

----*--------*--------*--------*--------*----
Cohort ID: 1	Cohort Treatment: 0.6
Cohort Participants:
120	124	119	108	
----*--------*--------*--------*--------*----
----*--------*--------*--------*--------*----
Cohort ID: 2	Cohort Treatment: 0.6
Cohort Participants:
102	112	115	103	
----*--------*--------*--------*--------*----
----*--------*--------*--------*--------*----
Cohort ID: 3	Cohort Treatment: 1
Cohort Participants:
104	113	114	106	
----*--------*--------*--------*--------*----
----*--------*--------*--------*--------*----
Cohort ID: 4	Cohort Treatment: 1
Cohort Participants:
122	109	107	111	
----*--------*--------*--------*--------*----
----*--------*--------*--------*--------*----
Cohort ID: 5	Cohort Treatment: 2
Cohort Participants:
123	105	110	117	
----*--------*--------*--------*--------*----
----*--------*--------*--------*--------*----
Cohort ID: 6	Cohort Treatment: 2
Cohort Participants:
101	118	116	121	
----*--------*--------*--------*--------*----


In [42]:
for cohort in cohorts:
	order = generate_game_order(cohort)
	for participant, opponent in order:
		game_id += 1
		print(f"Cohort: {cohort.id} - Treatment: {cohort.treatment} - Game {game_id}/{total_rounds}: {participant.id} vs {opponent.id}")

		if participant.rounds == 0:
			print(f"First round of participant {participant.id}")
			participant.set_prompt(generate_start_prompt(cohort.treatment))

		if opponent.rounds == 0:
			print(f"First round of participant {opponent.id}")
			opponent.set_prompt(generate_start_prompt(cohort.treatment))

		print(f"Prompting participant {participant.id}")
		ctx = prompt_model(participant.prompt, participant.memory)
		choice1 = get_choice(ctx[-1]['content'])
		print(f"Got response for participant {participant.id} with choice: {choice1}")
		participant.update_memory(ctx)

		print(f"Prompting participant {opponent.id}")
		ctx = prompt_model(opponent.prompt, opponent.memory)
		choice2 = get_choice(ctx[-1]['content'])
		print(f"Got response for participant {opponent.id} with choice: {choice2}")
		opponent.update_memory(ctx)

		participant.rounds += 1
		opponent.rounds += 1

		participant.payout = TREATMENTS[cohort.treatment][(choice1, choice2)][0]
		opponent.payout = TREATMENTS[cohort.treatment][(choice1, choice2)][1]

		print(f"Saving results to file")
		save_results('experiment.csv', {'game_id': [game_id], 'treatment': [cohort.treatment], 'cohort_id': [cohort.id], 'round': [participant.rounds], 'player1_id': [participant.id], 'player2_id': [opponent.id], 'player1_choice': [choice1], 'player2_choice': [choice2], 'player1_payout': [participant.payout], 'player2_payout': [opponent.payout]})
		print(f"Saved results to file")

		if participant.rounds == NUM_ROUNDS:
			participant.set_prompt(generate_end_prompt(choice1, choice2, cohort.treatment, participant.payout))
		else:
			participant.set_prompt(generate_cont_prompt(choice1, choice2, cohort.treatment))

		if opponent.rounds == NUM_ROUNDS:
			opponent.set_prompt(generate_end_prompt(choice1, choice2, cohort.treatment, opponent.payout))
		else:
			opponent.set_prompt(generate_cont_prompt(choice2, choice1, cohort.treatment))
		
		print('----*----'*10)

Cohort: 1 - Treatment: 0.6 - Game 1/36: 124 vs 120
First round of participant 124
First round of participant 120
Prompting participant 124


Got response for participant 124 with choice: B
Prompting participant 120
Got response for participant 120 with choice: R
Saving results to file
Saved results to file
----*--------*--------*--------*--------*--------*--------*--------*--------*--------*----
Cohort: 1 - Treatment: 0.6 - Game 2/36: 119 vs 108
First round of participant 119
First round of participant 108
Prompting participant 119
Got response for participant 119 with choice: R
Prompting participant 108
Got response for participant 108 with choice: R
Saving results to file
Saved results to file
----*--------*--------*--------*--------*--------*--------*--------*--------*--------*----
Cohort: 1 - Treatment: 0.6 - Game 3/36: 119 vs 120
Prompting participant 119
Got response for participant 119 with choice: R
Prompting participant 120
Got response for participant 120 with choice: B
Saving results to file
Saved results to file
----*--------*--------*--------*--------*--------*--------*--------*--------*--------*----
Cohort: 1 